In [2]:
import pandas as pd
df = pd.read_csv("../data/interim/df_repu.csv")

In [3]:
df["Texte_clean"].str.contains("des Républicains").value_counts()

Texte_clean
False    3654
True      131
Name: count, dtype: int64

In [4]:
df_des = df[df["Texte_clean"].str.contains("des Républicains")]

In [5]:
df_des

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,nombreMandats,experienceDepute,scoreParticipation,scoreParticipationSpecialite,scoreLoyaute,scoreMajorite,active,dateMaj,Texte_clean,repu_match_valide
1,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,2.0,3 ans,0.37,0.84,0.958,0.0,1.0,2025-06-24,Le groupe Rassemblement national ne cachera pa...,True
42,CRSANR5L16S2024O1N103,RUANR5L16S2024IDS28011,SCR5A2024O1,20240124140000000,mercredi 24 janvier 2024,1,103,AN,16,Session ordinaire 2023-2024,...,3.0,8 ans,0.16,0.37,0.952,0.0,1.0,2025-06-24,« Les représentants du peuple français […] ont...,True
46,CRSANR5L16S2024O1N103,RUANR5L16S2024IDS28011,SCR5A2024O1,20240124140000000,mercredi 24 janvier 2024,1,103,AN,16,Session ordinaire 2023-2024,...,2.0,3 ans,0.11,0.19,0.989,0.0,1.0,2025-06-24,Constitutionnaliser l’IVG est absolument indis...,True
66,CRSANR5L16S2024O1N088,RUANR5L16S2024IDS27957,SCR5A2024O1,20231219150000000,mardi 19 décembre 2023,1,88,AN,16,Session ordinaire 2023-2024,...,3.0,4 ans,0.07,NaN,0.912,0.0,0.0,2025-06-24,Vous ne nous ferez pas croire que vous êtes du...,True
74,CRSANR5L16S2023O1N013,RUANR5L16S2023IDS26389,SCR5A2023O1,20221011213000000,mardi 11 octobre 2022,2,13,AN,16,Session ordinaire 2022-2023,...,2.0,3 ans,0.31,0.58,0.935,0.0,1.0,2025-06-24,Nous les plaçons par ailleurs sous la coupe de...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3664,CRSANR5L16S2024O1N047,RUANR5L16S2024IDS27877,SCR5A2024O1,20231109220000000,jeudi 09 novembre 2023,Unique,47,AN,16,Session ordinaire 2023-2024,...,2.0,3 ans,0.29,0.15,0.789,0.0,1.0,2025-06-24,Nos économies financeront également une politi...,True
3665,CRSANR5L16S2024O1N053,RUANR5L16S2024IDS27677,SCR5A2024O1,20231115213000000,mercredi 15 novembre 2023,2,53,AN,16,Session ordinaire 2023-2024,...,2.0,3 ans,0.43,0.31,0.988,0.0,1.0,2025-06-24,La gauche dépose des motions de rejet préalabl...,True
3666,CRSANR5L16S2024O1N053,RUANR5L16S2024IDS27677,SCR5A2024O1,20231115213000000,mercredi 15 novembre 2023,2,53,AN,16,Session ordinaire 2023-2024,...,2.0,3 ans,0.26,0.68,0.999,0.0,1.0,2025-06-24,Votre pouvoir ne tient plus qu’à quelques fils...,True
3685,CRSANR5L16S2023O1N036,RUANR5L16S2023IDS26418,SCR5A2023O1,20221027150000000,jeudi 27 octobre 2022,2,36,AN,16,Session ordinaire 2022-2023,...,3.0,8 ans,0.10,0.03,0.866,0.0,1.0,2025-06-24,"Non, ce n’en est pas un, j’ai simplement deman...",True


In [ ]:
df_des["Texte_clean"].to_csv("temp_des_repu.txt", index=False, header=False)